# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
import pandas as pd
df = pd.read_csv('homepage_actions.csv')
print("Number of rows and columns:", df.shape)
print("\nColumn names:", df.columns)
print("\nData types of columns:")
print(df.dtypes)
print("\nSummary statistics:")
print(df.describe())

viewers = df[df['action'] == 'view']
clickers = df[df['action'] == 'click']
common_users = viewers[viewers['id'].isin(clickers['id'])]

print("\nNumber of viewers who also clicked:", len(common_users))

click_without_view = df[df['action'] == 'click'][~df['id'].isin(df[df['action'] == 'view']['id'])]

print("\nNumber of anomalies (click without view):", len(click_without_view))
control_group = df[df['group'] == 'control']['id'].unique()
experiment_group = df[df['group'] == 'experiment']['id'].unique()

overlap_users = set(control_group).intersection(experiment_group)

print("\nNumber of users in both control and experiment groups:", len(overlap_users))

Number of rows and columns: (8188, 4)

Column names: Index(['timestamp', 'id', 'group', 'action'], dtype='object')

Data types of columns:
timestamp    object
id            int64
group        object
action       object
dtype: object

Summary statistics:
                  id
count    8188.000000
mean   564699.749878
std    219085.845672
min    182988.000000
25%    373637.500000
50%    566840.500000
75%    758078.000000
max    937217.000000

Number of viewers who also clicked: 1860

Number of anomalies (click without view): 0

Number of users in both control and experiment groups: 0


C:\Users\hp\AppData\Local\Temp\ipykernel_23448\3553454404.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  click_without_view = df[df['action'] == 'click'][~df['id'].isin(df[df['action'] == 'view']['id'])]


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [2]:
from statsmodels.stats.proportion import proportions_ztest

control_clicks = len(df[(df['group'] == 'control') & (df['action'] == 'click')])
experiment_clicks = len(df[(df['group'] == 'experiment') & (df['action'] == 'click')])

control_users = len(df[df['group'] == 'control'])
experiment_users = len(df[df['group'] == 'experiment'])

control_ctr = control_clicks / control_users
experiment_ctr = experiment_clicks / experiment_users

z_stat, p_value = proportions_ztest([control_clicks, experiment_clicks], [control_users, experiment_users], alternative='larger')

alpha = 0.05

if p_value < alpha:
    print("The experimental homepage is more effective than the control group's homepage.")
else:
    print("There is no significant difference in effectiveness between the experimental and control groups' homepages.")

print("\nZ-statistic:", z_stat)
print("p-value:", p_value)

There is no significant difference in effectiveness between the experimental and control groups' homepages.

Z-statistic: -1.9332699751308076
p-value: 0.9733985257385523


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [3]:
control_ctr = control_clicks / control_users
expected_experiment_clicks = control_ctr * experiment_users
print('Expected number of clicks for the experiment group:', expected_experiment_clicks)

Expected number of clicks for the experiment group: 857.6848030018762


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [4]:
binomial_std_dev = (experiment_users * control_ctr * (1 - control_ctr)) ** 0.5
z_score = (experiment_clicks - expected_experiment_clicks) / binomial_std_dev
print('Z-score:', z_score)

Z-score: 2.71607302278784


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [5]:
from scipy.stats import norm
p_value_normal = 1 - norm.cdf(z_score)
print('p-value using the normal distribution:', p_value_normal)

p-value using the normal distribution: 0.0033030672759265522


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.